In [1]:

%matplotlib inline
import matplotlib.pyplot as plt
from mxnet import image

In [2]:
from mxnet import nd
import sys
sys.path.append('..')
import utils

def apply(img, aug, n=3):
    # 转成float，一是因为aug需要float类型数据来方便做变化。
    # 二是这里会有一次copy操作，因为有些aug直接通过改写输入
    #（而不是新建输出）获取性能的提升
    X = [aug(img.astype('float32')) for _ in range(n*n)]
    # 有些aug不保证输入是合法值，所以做一次clip
    # 显示浮点图片时imshow要求输入在[0,1]之间
    Y = nd.stack(*X).clip(0,255)/255
    utils.show_images(Y, n, n, figsize=(8,8))

In [3]:
def apply_aug_list(img, augs):
    for f in augs:
        img = f(img)
    return img 

In [4]:

train_augs = [
    image.HorizontalFlipAug(.5),
    image.RandomCropAug((28,28))
]

test_augs = [
    image.CenterCropAug((28,28))
]

In [5]:

from mxnet import gluon
from mxnet import nd
import sys
sys.path.append('..')
import utils

def get_transform(augs):
    def transform(data, label):
        # data: sample x height x width x channel
        # label: sample
        data = data.astype('float32')
        if augs is not None:
            # apply to each sample one-by-one and then stack
            data = nd.stack(*[
                apply_aug_list(d, augs) for d in data])
        data = nd.transpose(data, (0,3,1,2))
        return data, label.astype('float32')
    return transform

def get_data(batch_size, train_augs, test_augs=None):
    cifar10_train = gluon.data.vision.CIFAR10(
        train=True, transform=get_transform(train_augs))
    cifar10_test = gluon.data.vision.CIFAR10(
        train=False, transform=get_transform(test_augs))
    train_data = utils.DataLoader(
        cifar10_train, batch_size, shuffle=True)
    test_data = utils.DataLoader(
        cifar10_test, batch_size, shuffle=False)
    return (train_data, test_data)

In [6]:

from mxnet import init

def train(train_augs, test_augs, learning_rate=.1):
    batch_size = 128
    num_epochs = 1
    ctx = utils.try_all_gpus()
    loss = gluon.loss.SoftmaxCrossEntropyLoss()    
    train_data, test_data = get_data(
        batch_size, train_augs, test_augs)
    net = utils.resnet18(10) 
    net.initialize(ctx=ctx, init=init.Xavier())
    net.hybridize()
    trainer = gluon.Trainer(net.collect_params(),
                            'sgd', {'learning_rate': learning_rate})
    utils.train(
        train_data, test_data, net, loss, trainer, ctx, num_epochs)


使用增广：

In [7]:
# train(train_augs, test_augs)



不使用增广： 

In [8]:

# train(test_augs, test_augs)

# 调优测试

In [9]:

from mxnet import init

def train1(train_augs, test_augs, learning_rate=.1):
    batch_size = 128
    num_epochs =60
    ctx = utils.try_all_gpus()
    loss = gluon.loss.SoftmaxCrossEntropyLoss()    
    train_data, test_data = get_data(
        batch_size, train_augs, test_augs)
    net = utils.resnet18(10) 
    net.initialize(ctx=ctx, init=init.Xavier())
    net.hybridize()
    trainer = gluon.Trainer(net.collect_params(),
                            'sgd', {'learning_rate': learning_rate})
    utils.train(
        train_data, test_data, net, loss, trainer, ctx, num_epochs)
    
train1(train_augs, test_augs)

Start training on  [gpu(0)]
Epoch 0. Loss: 1.478, Train acc 0.47, Test acc 0.56, Time 33.4 sec
Epoch 1. Loss: 1.048, Train acc 0.63, Test acc 0.62, Time 34.0 sec
Epoch 2. Loss: 0.868, Train acc 0.69, Test acc 0.67, Time 31.7 sec
Epoch 3. Loss: 0.752, Train acc 0.74, Test acc 0.73, Time 34.1 sec
Epoch 4. Loss: 0.676, Train acc 0.76, Test acc 0.75, Time 34.1 sec
Epoch 5. Loss: 0.617, Train acc 0.78, Test acc 0.76, Time 33.5 sec
Epoch 6. Loss: 0.576, Train acc 0.80, Test acc 0.77, Time 32.7 sec
Epoch 7. Loss: 0.536, Train acc 0.81, Test acc 0.78, Time 33.7 sec
Epoch 8. Loss: 0.503, Train acc 0.82, Test acc 0.80, Time 33.6 sec
Epoch 9. Loss: 0.476, Train acc 0.83, Test acc 0.81, Time 35.9 sec
Epoch 10. Loss: 0.449, Train acc 0.84, Test acc 0.81, Time 33.1 sec
Epoch 11. Loss: 0.423, Train acc 0.85, Test acc 0.81, Time 33.6 sec
Epoch 12. Loss: 0.404, Train acc 0.86, Test acc 0.82, Time 28.2 sec
Epoch 13. Loss: 0.379, Train acc 0.87, Test acc 0.83, Time 32.4 sec
Epoch 14. Loss: 0.360, Train a

Start training on  [gpu(0)]
Epoch 0. Loss: 1.487, Train acc 0.47, Test acc 0.55, Time 21.8 sec
Epoch 1. Loss: 1.058, Train acc 0.63, Test acc 0.65, Time 21.1 sec
Epoch 2. Loss: 0.875, Train acc 0.69, Test acc 0.70, Time 20.5 sec
Epoch 3. Loss: 0.754, Train acc 0.74, Test acc 0.74, Time 21.2 sec
Epoch 4. Loss: 0.678, Train acc 0.76, Test acc 0.76, Time 19.9 sec
Epoch 5. Loss: 0.622, Train acc 0.79, Test acc 0.75, Time 21.3 sec
Epoch 6. Loss: 0.569, Train acc 0.80, Test acc 0.78, Time 19.6 sec
Epoch 7. Loss: 0.535, Train acc 0.81, Test acc 0.78, Time 21.8 sec
Epoch 8. Loss: 0.498, Train acc 0.83, Test acc 0.77, Time 19.8 sec
Epoch 9. Loss: 0.472, Train acc 0.84, Test acc 0.80, Time 21.2 sec
Epoch 10. Loss: 0.450, Train acc 0.84, Test acc 0.81, Time 20.0 sec
Epoch 11. Loss: 0.420, Train acc 0.85, Test acc 0.80, Time 20.6 sec
Epoch 12. Loss: 0.402, Train acc 0.86, Test acc 0.81, Time 19.9 sec
Epoch 13. Loss: 0.377, Train acc 0.87, Test acc 0.83, Time 20.3 sec
Epoch 14. Loss: 0.366, Train acc 0.87, Test acc 0.82, Time 20.6 sec
Epoch 15. Loss: 0.343, Train acc 0.88, Test acc 0.82, Time 21.7 sec
Epoch 16. Loss: 0.327, Train acc 0.89, Test acc 0.83, Time 21.4 sec
Epoch 17. Loss: 0.316, Train acc 0.89, Test acc 0.82, Time 20.3 sec
Epoch 18. Loss: 0.302, Train acc 0.89, Test acc 0.82, Time 21.9 sec
Epoch 19. Loss: 0.281, Train acc 0.90, Test acc 0.84, Time 19.7 sec
Epoch 20. Loss: 0.277, Train acc 0.90, Test acc 0.83, Time 20.0 sec
Epoch 21. Loss: 0.259, Train acc 0.91, Test acc 0.84, Time 20.7 sec
Epoch 22. Loss: 0.250, Train acc 0.91, Test acc 0.84, Time 20.8 sec
Epoch 23. Loss: 0.242, Train acc 0.91, Test acc 0.85, Time 19.8 sec
Epoch 24. Loss: 0.230, Train acc 0.92, Test acc 0.83, Time 19.8 sec
Epoch 25. Loss: 0.225, Train acc 0.92, Test acc 0.85, Time 21.2 sec
Epoch 26. Loss: 0.205, Train acc 0.93, Test acc 0.84, Time 21.4 sec
Epoch 27. Loss: 0.202, Train acc 0.93, Test acc 0.85, Time 21.7 sec
Epoch 28. Loss: 0.200, Train acc 0.93, Test acc 0.82, Time 21.8 sec
Epoch 34. Loss: 0.158, Train acc 0.94, Test acc 0.84, Time 20.6 sec
Epoch 35. Loss: 0.150, Train acc 0.95, Test acc 0.85, Time 20.6 sec
Epoch 36. Loss: 0.147, Train acc 0.95, Test acc 0.86, Time 20.1 sec
Epoch 37. Loss: 0.141, Train acc 0.95, Test acc 0.85, Time 20.5 sec
Epoch 38. Loss: 0.133, Train acc 0.95, Test acc 0.85, Time 20.2 sec
Epoch 39. Loss: 0.130, Train acc 0.95, Test acc 0.85, Time 19.9 sec
Epoch 40. Loss: 0.132, Train acc 0.95, Test acc 0.85, Time 21.2 sec
Epoch 41. Loss: 0.124, Train acc 0.96, Test acc 0.85, Time 19.6 sec
Epoch 42. Loss: 0.120, Train acc 0.96, Test acc 0.85, Time 21.7 sec
Epoch 43. Loss: 0.113, Train acc 0.96, Test acc 0.86, Time 21.4 sec
Epoch 44. Loss: 0.106, Train acc 0.96, Test acc 0.86, Time 21.3 sec
Epoch 45. Loss: 0.105, Train acc 0.96, Test acc 0.85, Time 21.5 sec
Epoch 46. Loss: 0.103, Train acc 0.96, Test acc 0.86, Time 20.5 sec
Epoch 47. Loss: 0.104, Train acc 0.96, Test acc 0.86, Time 20.0 sec
Epoch 48. Loss: 0.094, Train acc 0.97, Test acc 0.85, Time 20.4 sec
Epoch 49. Loss: 0.098, Train acc 0.97, Test acc 0.85, Time 20.3 sec
Epoch 50. Loss: 0.090, Train acc 0.97, Test acc 0.85, Time 21.8 sec
Epoch 51. Loss: 0.093, Train acc 0.97, Test acc 0.86, Time 21.0 sec
Epoch 52. Loss: 0.088, Train acc 0.97, Test acc 0.86, Time 21.3 sec
Epoch 53. Loss: 0.084, Train acc 0.97, Test acc 0.86, Time 21.1 sec
Epoch 54. Loss: 0.081, Train acc 0.97, Test acc 0.87, Time 19.8 sec
Epoch 55. Loss: 0.081, Train acc 0.97, Test acc 0.86, Time 21.1 sec
Epoch 56. Loss: 0.076, Train acc 0.97, Test acc 0.86, Time 19.9 sec
Epoch 57. Loss: 0.075, Train acc 0.97, Test acc 0.86, Time 19.9 sec
Epoch 58. Loss: 0.068, Train acc 0.98, Test acc 0.86, Time 20.3 sec
Epoch 59. Loss: 0.077, Train acc 0.97, Test acc 0.86, Time 20.0 sec